In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps

file_path = '..\Resources\California_PPP.csv'

california_df=pd.read_csv(file_path)
california_df.shape

(1302241, 23)

In [43]:
#this code takes the county and city from cdtfa and cleans it removing special characters and ()


url ='https://www.cdtfa.ca.gov/taxes-and-fees/rates.aspx'

california_counties_pull = pd.read_html(url, header=0)
california_counties = california_counties_pull[0]
california_counties = california_counties[['Location', 'County']]
california_counties['Location'] = california_counties['Location'].str.replace('*','', regex = False)
california_counties['Location'] = california_counties['Location'].str.replace(r"\(.*\)","")
california_counties['Location']  = california_counties['Location'].str.strip()
california_counties=california_counties.rename(columns={'Location': 'BorrowerCity'})
california_counties


#this cleans the city column to prepare for merge

california_df.columns

title_case = california_df['BorrowerCity'].str.title()
california_df['BorrowerCity'] = title_case
california_df['BorrowerCity'] = california_df['BorrowerCity'].replace(
    {'City Of Industry': 'City of Industry','Carmel':'Carmel-by-the-Sea', 'Carmel-By-The-Sea': 'Carmel-by-the-Sea',
    'Huntingtn Bch':'Huntington Beach','Rch Cucamonga':'Rancho Cucamonga','N Hollywood':'North Hollywood','La Canada Flintridge':'La Canada- Flintridge',
    'Woodland Hls':'Woodland Hills','La Canada':'La Canada- Flintridge','Playa Vista':'Los Angeles','Newport Coast':'Newport Beach'})



<ipython-input-43-e4cd8638ef09>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  california_counties['Location'] = california_counties['Location'].str.replace('*','', regex = False)
<ipython-input-43-e4cd8638ef09>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  california_counties['Location'] = california_counties['Location'].str.replace(r"\(.*\)","")
<ipython-input-43-e4cd8638ef09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  california_counties['Loc

In [72]:
#this merges the county table from cdtfa with our ppp data set

california_df_merge = pd.merge(california_df, california_counties, on="BorrowerCity", how="left")

#this finds the rows that does not have counties, there are numerous miss spellings so we cleaned the data down to less than 1% of the total data set
pd.isna(california_df_merge['County']).sum()
df_no_county = california_df_merge.loc[california_df_merge['County'].isna()]
print(df_no_county["LoanNumber"].count())
df_bin = df_no_county.groupby("BorrowerCity")
df_bin.count().sort_values("LoanNumber", ascending=False).head(10)


12737


,LoanNumber,DateApproved,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerState,BorrowerZip,CurrentApprovalAmount,RuralUrbanIndicator,HubzoneIndicator,...,NAICSCode,Race,PAYROLL_PROCEED,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,County
BorrowerCity,,,,,,,,,,,,,,,,,,,,,
Mckinleyville,300,300,300,300,300,300,300,300,300,300,...,297,300,300,300,300,300,300,300,300,0
W Hollywood,279,279,279,279,279,279,279,279,279,279,...,268,279,279,279,279,279,279,279,279,0
Sn Bernrdno,259,259,259,259,259,259,259,259,259,259,...,248,259,259,259,259,259,259,259,259,0
Oak Hills,228,228,228,228,228,228,228,228,228,228,...,225,228,228,228,228,228,228,228,228,0
Westlake Vlg,226,226,226,226,226,226,226,226,226,226,...,216,226,226,226,226,226,226,226,226,0
S Lake Tahoe,218,218,218,218,218,218,218,218,218,218,...,195,218,218,218,218,218,218,218,218,0
Lake Balboa,216,216,216,216,216,216,216,216,216,216,...,208,216,216,216,216,216,216,216,216,0
S San Fran,214,214,214,214,214,214,214,214,214,214,...,212,214,214,214,214,214,214,214,214,0
Cardiff,212,212,212,212,212,212,212,212,212,212,...,200,212,211,212,212,212,212,212,212,0


In [71]:
#This creates the county data by PPP and PPS
california_df_county = california_df_merge[['LoanNumber','CurrentApprovalAmount','County','ProcessingMethod']]

california_df_county_ppp = california_df_county.loc[california_df_county["ProcessingMethod"]=="PPP"]
california_df_county_pps = california_df_county.loc[california_df_county["ProcessingMethod"]=="PPS"]




In [114]:
#This is PPP county Data

county_group_ppp = california_df_county_ppp.groupby('County')
ppp_loan_count = county_group_ppp['CurrentApprovalAmount'].count()
ppp_loan_amount = county_group_ppp['CurrentApprovalAmount'].sum()/1000000

total_loan_count_ppp = ppp_loan_count.sum()
total_loan_sum_ppp = ppp_loan_amount.sum()


ppp_county = pd.merge(ppp_loan_count, ppp_loan_amount, on="County", how="left")
ppp_county=ppp_county.rename(columns={'CurrentApprovalAmount_x': 'Loan Count','CurrentApprovalAmount_y': 'Loan Amount (Millions)'})

ppp_county["Count %"] = ppp_county["Loan Count"]/total_loan_count_ppp
ppp_county["Amount %"] = ppp_county["Loan Amount (Millions)"]/total_loan_sum_ppp
ppp_county.sort_values("Loan Amount (Millions)", ascending=False)

,Loan Count,Loan Amount (Millions),Count %,Amount %
County,,,,
Los Angeles,323117,21474.230487,0.343104,0.287401
Orange,87502,8343.040584,0.092915,0.111659
San Diego,74351,6435.608582,0.078950,0.086131
Alameda,41089,4056.945323,0.043631,0.054296
Santa Clara,36750,4015.774333,0.039023,0.053745
San Francisco,30035,3404.610667,0.031893,0.045566
Riverside,44785,2812.927343,0.047555,0.037647
San Bernardino,44993,2692.858845,0.047776,0.036040
Sacramento,32120,2190.670012,0.034107,0.029319


In [113]:
#This is PPS county Data

county_group_pps = california_df_county_pps.groupby('County')
pps_loan_count = county_group_pps['CurrentApprovalAmount'].count()
pps_loan_amount = county_group_pps['CurrentApprovalAmount'].sum()/1000000

total_loan_count_pps = pps_loan_count.sum()
total_loan_sum_pps = pps_loan_amount.sum()


pps_county = pd.merge(pps_loan_count, pps_loan_amount, on="County", how="left")
pps_county=pps_county.rename(columns={'CurrentApprovalAmount_x': 'Loan Count','CurrentApprovalAmount_y': 'Loan Amount (Millions)'})

pps_county["Count %"] = pps_county["Loan Count"]/total_loan_count_pps
pps_county["Amount %"] = pps_county["Loan Amount (Millions)"]/total_loan_sum_pps
pps_county.sort_values("Loan Amount (Millions)", ascending=False)


,Loan Count,Loan Amount (Millions),Count %,Amount %
County,,,,
Los Angeles,123793,8903.289006,0.349080,0.308090
Orange,36278,3266.553838,0.102299,0.113036
San Diego,28960,2512.439561,0.081663,0.086941
Santa Clara,15903,1561.953556,0.044844,0.054050
Alameda,16590,1526.182044,0.046782,0.052812
San Francisco,13291,1403.370639,0.037479,0.048562
Riverside,14600,1049.402431,0.041170,0.036314
San Bernardino,13724,913.583043,0.038700,0.031614
San Mateo,7532,770.520095,0.021239,0.026663
